In [ ]:
!git clone https://github.com/EulerMacaroni/Speculative-Decoding

Cloning into 'Speculative-Decoding'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 296 (delta 51), reused 50 (delta 45), pack-reused 234 (from 1)
Receiving objects: 100% (296/296), 2.07 MiB | 4.26 MiB/s, done.
Resolving deltas: 100% (161/161), done.


In [ ]:
import sys
sys.path.append("/content/Speculative-Decoding")

In [ ]:
from huggingface_hub import login
login(token="")


In [ ]:
# from google.colab import files
# uploaded = files.upload()


In [ ]:
!pip install \
  tqdm \
  tokenizers \
  torch \
  transformers \
  accelerate \
  sentence_transformers \
  scikit-learn \
  bitsandbytes \
  # --extra-index-url https://download.pytorch.org/whl/cu118


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from ngram_assisted.ngram_assisted import ngram_assisted_speculative_generate
from ngram_assisted.ngram_storage import NGramStorage
from sampling.base_decoding import autoregressive_generate
from sampling.speculative_decoding import speculative_generate
from utils.logits_processor import TwoStageSamplingProcessor

In [ ]:
import os
os.listdir("/content/Speculative-Decoding")

['requirements.txt',
 'infer.py',
 'utils',
 'data',
 'README.md',
 'test_optimized_for_colab.ipynb',
 'results',
 '.gitignore',
 'LICENSE',
 'sampling',
 'test.py',
 'ngram_assisted',
 '.git']

In [ ]:
import random
import re
import string
import time
from collections import Counter

import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig
from transformers import BitsAndBytesConfig



In [ ]:

def normalize_answer(s):
    """Lower text and remove punctuation, articles, and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        return "".join(ch for ch in text if ch not in string.punctuation)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    """Calculate F1 score between prediction and ground truth."""
    pred_tokens = normalize_answer(prediction).split()
    gt_tokens = normalize_answer(ground_truth).split()
    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    return 2 * precision * recall / (precision + recall)


def exact_match(prediction, ground_truth):
    """Check if prediction matches ground truth exactly."""
    return normalize_answer(prediction) == normalize_answer(ground_truth)


def embedding_similarity(pred, truth, model):
    emb1 = model.encode([pred], convert_to_tensor=True)
    emb2 = model.encode([truth], convert_to_tensor=True)

    # Move to CPU for sklearn
    if emb1.device.type != "cpu":
        emb1 = emb1.cpu()
        emb2 = emb2.cpu()

    return float(cosine_similarity(emb1.numpy(), emb2.numpy())[0][0])


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)


In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install -U bitsandbytes

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)


bnb_config = BitsAndBytesConfig(load_in_8bit=True)
target_model = "meta-llama/Llama-2-7b-chat-hf"
# target_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
draft_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# target_quantize = QuantoConfig(weights="int8")
# drafter_quantize = QuantoConfig(weights="int8")
tokenizer = AutoTokenizer.from_pretrained(target_model,
                                          trust_remote_code=True,
                                          # use_auth_token=True
                                          )
drafter = AutoModelForCausalLM.from_pretrained(
    draft_model,
    # quantization_config=drafter_quantize,
    device_map=device,
    trust_remote_code=True,
    # use_auth_token=True,
    # torch_dtype=torch.float16,
    quantization_config=bnb_config
)
target = AutoModelForCausalLM.from_pretrained(
    target_model,
    # quantization_config=target_quantize,
    device_map=device,
    trust_remote_code=True,
    # use_auth_token=True,
    # torch_dtype=torch.float16,
    quantization_config=bnb_config
)
drafter.eval()
target.eval()

end_tokens = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:

df = pd.read_csv("Speculative-Decoding/data/QAwithTemp.csv")
# df = df.head(1)

processors = {
    "greedy": lambda temp: GreedyProcessor(temperature=temp),
    "multinomial": lambda temp: MultinomialProcessor(temperature=temp),
    "topk": lambda temp: TopKProcessor(temperature=temp, top_k=10),
    "nucleus": lambda temp: NucleusProcessor(temperature=temp, top_p=0.95),
    "topknucleus": lambda temp: TopKNucleusProcessor(
        temperature=temp, top_k=10, top_p=0.95
    ),
    "TwoStage": lambda temp: TwoStageSamplingProcessor(
        temperature=temp, top_k=10, noise_scale=0.4
    )
}

model_types = ["speculative", "ngram", "autoregressive"]

all_rows = []

for proc_name, proc_builder in processors.items():
    for idx, row in df.iterrows():
        prompt = tokenizer.apply_chat_template(
            [{"role": "user", "content": row["question"]}],
            add_generation_prompt=True,
            tokenize=False,
        )
        answer = row["answer"]
        temp = 0.8
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids[0].tolist()
        max_len = 40

        for model_name in model_types:
            processor = (
                proc_builder(temp) if proc_name != "greedy" else GreedyProcessor()
            )

            try:
                start = time.time()
                if model_name == "speculative":
                    output_ids, acc_rate = speculative_generate(
                        inputs=input_ids,
                        drafter=drafter,
                        target=target,
                        tokenizer=tokenizer,
                        logits_processor=processor,
                        gamma=4,
                        max_gen_len=max_len,
                        eos_tokens_id=end_tokens,
                        use_cache=False,
                        debug=False,
                    )
                elif model_name == "ngram":
                    ngram = NGramStorage(n=3, vocab_size=target.config.vocab_size)
                    ngram.reset()
                    output_ids, acc_rate = ngram_assisted_speculative_generate(
                        inputs=input_ids,
                        ngramstorage=ngram,
                        target=target,
                        tokenizer=tokenizer,
                        gamma=4,
                        filler_top_k=3,
                        logits_processor=processor,
                        max_gen_len=max_len,
                        eos_tokens_id=end_tokens,
                        use_cache=False,
                        debug=False,
                        stop_if_unknown=True,
                    )
                else:
                    output_ids = autoregressive_generate(
                        inputs=input_ids,
                        model=target,
                        logits_processor=processor,
                        max_gen_len=max_len,
                        eos_tokens_id=end_tokens,
                        use_cache=False,
                        debug=False,
                    )
                    acc_rate = None

                duration = time.time() - start
                tps = len(output_ids) / duration if duration > 0 else float("inf")
                output_text = tokenizer.decode(output_ids, skip_special_tokens=True)
                # f1 = f1_score(output_text, answer)
                # em = exact_match(output_text, answer)
                sim = embedding_similarity(output_text, answer, embedding_model)
                # score = 0.6 * f1 + 0.4 * em
                score = sim
                # print(score)

                all_rows.append(
                    {
                        "question": row["question"],
                        "answer": answer,
                        "temperature": temp,
                        "model": model_name,
                        "processor": proc_name,
                        "output": output_text,
                        "score": score,
                        "acceptance_rate": acc_rate,
                        "tokens_per_sec": round(tps, 2),
                    }
                )

                print(
                    f"Model: {model_name}, Processor: {proc_name}, Score: {score}, tokens/s: {tps}, Acceptance Rate: {acc_rate}"
                )
            except Exception as e:
                print(
                    f" Error in {model_name} with processor {proc_name} on question: '{row['question']}'\n{e}"
                )
                continue

# Save to CSV
final_df = pd.DataFrame(all_rows)
# print(final_df)
# import os
import os
os.makedirs("results", exist_ok=True)

final_df.to_csv("results/dynamictemp.csv", index=False)
print("Evaluation complete. Results saved to results/all_model_results.csv")


Model: speculative, Processor: TwoStage, Score: 0.5668257474899292, tokens/s: 5.153862012151555, Acceptance Rate: 0.4807692307692308
Model: ngram, Processor: TwoStage, Score: 0.5531858801841736, tokens/s: 7.91140845589799, Acceptance Rate: 0.5833333333333334
Model: autoregressive, Processor: TwoStage, Score: 0.5188355445861816, tokens/s: 6.437720527557056, Acceptance Rate: None
Model: speculative, Processor: TwoStage, Score: 0.1457817554473877, tokens/s: 4.303774816225512, Acceptance Rate: 0.3333333333333333
Model: ngram, Processor: TwoStage, Score: 0.15392223000526428, tokens/s: 8.78627249662932, Acceptance Rate: 0.45
Model: autoregressive, Processor: TwoStage, Score: 0.17185965180397034, tokens/s: 6.386031530458204, Acceptance Rate: None
Model: speculative, Processor: TwoStage, Score: 0.3954220414161682, tokens/s: 4.581654146376845, Acceptance Rate: 0.375
Model: ngram, Processor: TwoStage, Score: 0.3954220414161682, tokens/s: 9.250429846948903, Acceptance Rate: 0.3333333333333333
Mod

In [ ]:
# Define prompts manually
# prompts = [
#     "What are the benefits of daily meditation on mental health?",
#     "Explain how black holes are formed in simple terms.",
#     "What are some practical applications of reinforcement learning?"
# ]

prompts = ["Artificial intelligence has transformed the way we " ]
# Experiment config
noise_scales = [0.0, 0.3, 0.6, 0.9]
model_types = ["speculative", "ngram", "autoregressive"]
temperature = 0.8
top_k = 10
max_len = 200
all_rows = []

# Run experiment
for prompt_text in prompts:
    # prompt = tokenizer.apply_chat_template(
    #     [{"role": "user", "content": prompt_text}],
    #     add_generation_prompt=True,
    #     tokenize=False,
    # )
    prompt= tokenizer.apply_chat_template(
    [
    {"role": "user", "content": "Finish this sentence: Artificial intelligence has transformed the way we"}
    ],
    add_generation_prompt=True,
    tokenize=False,
    )
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids[0].tolist()
    # fake_answer = "N/A"  # No gold answer available

    for noise in noise_scales:
        processor = TwoStageSamplingProcessor(
            temperature=temperature, top_k=top_k, noise_scale=noise
        )

        for model_name in model_types:
            try:
                start = time.time()

                if model_name == "speculative":
                    output_ids, acc_rate = speculative_generate(
                        inputs=input_ids,
                        drafter=drafter,
                        target=target,
                        tokenizer=tokenizer,
                        logits_processor=processor,
                        gamma=4,
                        max_gen_len=max_len,
                        eos_tokens_id=end_tokens,
                        use_cache=False,
                        debug=False,
                    )
                elif model_name == "ngram":
                    ngram = NGramStorage(n=3, vocab_size=target.config.vocab_size)
                    ngram.reset()
                    output_ids, acc_rate = ngram_assisted_speculative_generate(
                        inputs=input_ids,
                        ngramstorage=ngram,
                        target=target,
                        tokenizer=tokenizer,
                        gamma=4,
                        filler_top_k=3,
                        logits_processor=processor,
                        max_gen_len=max_len,
                        eos_tokens_id=end_tokens,
                        use_cache=False,
                        debug=False,
                        stop_if_unknown=True,
                    )
                else:
                    output_ids = autoregressive_generate(
                        inputs=input_ids,
                        model=target,
                        logits_processor=processor,
                        max_gen_len=max_len,
                        eos_tokens_id=end_tokens,
                        use_cache=False,
                        debug=False,
                    )
                    acc_rate = None

                duration = time.time() - start
                tps = len(output_ids) / duration if duration > 0 else float("inf")
                output_text = tokenizer.decode(output_ids, skip_special_tokens=True)

                # sim_score = embedding_similarity(output_text, fake_answer)  # Placeholder metric

                all_rows.append({
                    "prompt": prompt_text,
                    "output": output_text,
                    "temperature": temperature,
                    "model": model_name,
                    "processor": "TopKwNoise",
                    "noise_scale": noise,
                    # "score": sim_score,
                    "acceptance_rate": acc_rate,
                    "tokens_per_sec": round(tps, 2),
                })

                print(f"[{model_name} | noise={noise}] tps: {tps:.2f}, accept: {acc_rate}")
            except Exception as e:
                print(f"❌ Error [{model_name}, noise={noise}] on prompt: '{prompt_text}'\n{e}")
                continue

# Save results
os.makedirs("results", exist_ok=True)
out_path = "results/twostage_noise_manual_prompts2.csv"
pd.DataFrame(all_rows).to_csv(out_path, index=False)
print(f"\n Done. Results saved to: {out_path}")

[speculative | noise=0.0] tps: 4.28, accept: 0.3961038961038961
[ngram | noise=0.0] tps: 5.73, accept: 0.22727272727272727
[autoregressive | noise=0.0] tps: 5.45, accept: None
[speculative | noise=0.3] tps: 4.35, accept: 0.40728476821192056
[ngram | noise=0.3] tps: 5.73, accept: 0.1875
[autoregressive | noise=0.3] tps: 5.50, accept: None
[speculative | noise=0.6] tps: 4.51, accept: 0.4175084175084175
[ngram | noise=0.6] tps: 5.91, accept: 0.2222222222222222
[autoregressive | noise=0.6] tps: 5.47, accept: None
[speculative | noise=0.9] tps: 4.26, accept: 0.38461538461538464
[ngram | noise=0.9] tps: 6.03, accept: 0.27631578947368424
[autoregressive | noise=0.9] tps: 5.49, accept: None

 Done. Results saved to: results/twostage_noise_manual_prompts2.csv
